In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import requests
import os
import sqlite3
from contextlib import contextmanager

In [2]:
token = "2528|6LQUYU9ZOJdvF24GGo4hTMKKCghaM6pk5yYtCarj"
headers = {
    "Accept": "application/json",
    "Authorization": f"Bearer {token}"
}
url = "https://restfulcountries.com/api/v1/countries"
response = requests.get(url, headers=headers)
data1 = None
if response.status_code == 200:
    data = response.json()
    data1 = data
else:
    print(f"Erro na requisição: {response.status_code}")
    print(response.text)

print('Funciona!')


Funciona!


In [3]:
def alerta(url, token):
    headers = {
        "Accept": "application/json",
        "Authorization": f"Bearer {token}"
    }
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.status_code
        else:
            print(f"Erro na requisição: Status Code {response.status_code}")
            print(response.text)
            return None
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar a URL: {e}")
        return None

token = "2528|6LQUYU9ZOJdvF24GGo4hTMKKCghaM6pk5yYtCarj"
url = "https://restfulcountries.com/api/v1/countries"
status_code = alerta(url, token)

if status_code == 200:
    print(f"O site está disponível (Status Code: {status_code}).")
elif status_code is None:
    print("O site pode estar indisponível ou houve um erro na requisição.")


O site está disponível (Status Code: 200).


In [4]:
def obter_dados_da_api(url, token):
    try:
        response = requests.get(url, token)
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar a API: {e}")
        return None


In [5]:
def criar_dataframe(data):
    if data is None:
        return None
    
    paises = []
    for pais in data['data']:
        pais_info = {
            'Nome Comum': pais['name'],
            'Nome Oficial': pais['full_name'],
            'Código Alpha-2': pais['iso2'],
            'Capital': pais['capital']
        }
        paises.append(pais_info)
    
    return pd.DataFrame(paises)

In [6]:
def criar_dataframe_populacao(data):
    if data is None:
        return None
    
    paises = []
    for pais in data['data']:
        pais_info = {
            'Nome Comum': pais['name'],
            'População': pais['covid19']['total_case'],
            'Área': pais['size'],
            'Continente': pais['continent']
        }
        paises.append(pais_info)
    
    return pd.DataFrame(paises)


In [7]:
def criar_dataframe_membro_da_onu(data):
    if data is None:
        return None
    
    paises = []
    for pais in data['data']:
        pais_info = {
            'Nome Comum': pais.get('name'),
                        'Nome Oficial': pais.get('full_name'),
                        'Código Alpha-2': pais.get('iso2'),
                        'Capital': pais.get('capital'),
                        'População': pais.get('population'),
                        'Área': pais.get('size'),
                        'Continente': pais.get('continent'),
                        'Independente': pais.get('independent'),
                        'Membro da ONU': pais.get('unMember'),
                        'Status': pais.get('status'), 
        }
        paises.append(pais_info)
    
    return pd.DataFrame(paises)

In [8]:
df_paises = criar_dataframe(data)
df_populacao = criar_dataframe_populacao(data)
df_onu = criar_dataframe_membro_da_onu(data)

##Salva DataFrames do Pandas em arquivos CSV, mas apenas se esses DataFrames específicos já existirem na memória do programa e não forem None

In [9]:
db_path = os.path.join("C:\\Users\\EberPC\\Documents\\andrehbahia", "dados_paises.db")

In [10]:
endereco_base = "C:\\Users\\EberPC\\Documents\\andrehbahia"
db_path = os.path.join(endereco_base, "dados_analise.db")

try:
    # Conecta ao banco de dados (cria o arquivo se não existir)
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    if 'df_paises' in locals() and df_paises is not None:
        nome_tabela_paises = "paises"
        df_paises.to_sql(nome_tabela_paises, conn, if_exists='replace', index=False)
        print(f"DataFrame 'df_paises' salvo na tabela '{nome_tabela_paises}' em '{db_path}'.")

    if 'df_populacao' in locals() and df_populacao is not None:
        nome_tabela_populacao = "populacao"
        df_populacao.to_sql(nome_tabela_populacao, conn, if_exists='replace', index=False)
        print(f"DataFrame 'df_populacao' salvo na tabela '{nome_tabela_populacao}' em '{db_path}'.")

    if 'df_onu' in locals() and df_onu is not None:
        nome_tabela_onu = "onu"
        df_onu.to_sql(nome_tabela_onu, conn, if_exists='replace', index=False)
        print(f"DataFrame 'df_onu' salvo na tabela '{nome_tabela_onu}' em '{db_path}'.")

    # Commit as alterações
    conn.commit()

except sqlite3.Error as e:
    print(f"Erro ao interagir com o banco de dados SQLite: {e}")

finally:
    # Fecha a conexão
    if conn:
        conn.close()

DataFrame 'df_paises' salvo na tabela 'paises' em 'C:\Users\EberPC\Documents\andrehbahia\dados_analise.db'.
DataFrame 'df_populacao' salvo na tabela 'populacao' em 'C:\Users\EberPC\Documents\andrehbahia\dados_analise.db'.
DataFrame 'df_onu' salvo na tabela 'onu' em 'C:\Users\EberPC\Documents\andrehbahia\dados_analise.db'.


In [11]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
for table in tables:
    print(table[0])
conn.close()

membros_transitorio
paises
populacao
onu


In [12]:
conn = sqlite3.connect(db_path)
df_paises = pd.read_sql_query(f"SELECT * FROM {nome_tabela_paises}", conn)
conn.close()

df_paises.head()

,Nome Comum,Nome Oficial,Código Alpha-2,Capital
0,Afghanistan,Islamic Republic of Afghanistan,AF,Kabul
1,Albania,Republic of Albania | Shqipëria,AL,Tirana
2,Algeria,Democratic Republic of Algeria,DZ,Algiers
3,Andorra,Principality of Andorra,AD,Andorra la Vella
4,Angola,Republic of Angola,AO,Luanda


In [13]:
df_paises.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Nome Comum      186 non-null    object
 1   Nome Oficial    184 non-null    object
 2   Código Alpha-2  186 non-null    object
 3   Capital         186 non-null    object
dtypes: object(4)
memory usage: 5.9+ KB


In [14]:
df_paises.shape

(186, 4)

In [15]:
df_paises.columns

Index(['Nome Comum', 'Nome Oficial', 'Código Alpha-2', 'Capital'], dtype='object')

In [16]:
conn = sqlite3.connect(db_path)
df_populacao = pd.read_sql_query(f"SELECT * FROM {nome_tabela_populacao}", conn)
conn.close()
df_populacao.head()

,Nome Comum,População,Área,Continente
0,Afghanistan,"46,498","652,000 km²",Asia
1,Albania,"37,625","28,748 km²",Europe
2,Algeria,"83,199","2,381,741 km²",Africa
3,Andorra,"6,712",468 km²,Europe
4,Angola,"15,139","1,246,620 km²",Africa


In [17]:
df_populacao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Nome Comum  186 non-null    object
 1   População   186 non-null    object
 2   Área        186 non-null    object
 3   Continente  184 non-null    object
dtypes: object(4)
memory usage: 5.9+ KB


In [18]:
df_populacao.shape

(186, 4)

In [19]:
df_populacao.columns

Index(['Nome Comum', 'População', 'Área', 'Continente'], dtype='object')

In [20]:
conn = sqlite3.connect(db_path)
df_onu = pd.read_sql_query(f"SELECT * FROM {nome_tabela_onu}", conn)
conn.close()
df_onu.head()

,Nome Comum,Nome Oficial,Código Alpha-2,Capital,População,Área,Continente,Independente,Membro da ONU,Status
0,Afghanistan,Islamic Republic of Afghanistan,AF,Kabul,"39,306,195","652,000 km²",Asia,None,None,None
1,Albania,Republic of Albania | Shqipëria,AL,Tirana,"2,876,490","28,748 km²",Europe,None,None,None
2,Algeria,Democratic Republic of Algeria,DZ,Algiers,"44,190,030","2,381,741 km²",Africa,None,None,None
3,Andorra,Principality of Andorra,AD,Andorra la Vella,"77,317",468 km²,Europe,None,None,None
4,Angola,Republic of Angola,AO,Luanda,"33,312,843","1,246,620 km²",Africa,None,None,None


In [21]:
df_onu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Nome Comum      186 non-null    object
 1   Nome Oficial    184 non-null    object
 2   Código Alpha-2  186 non-null    object
 3   Capital         186 non-null    object
 4   População       186 non-null    object
 5   Área            186 non-null    object
 6   Continente      184 non-null    object
 7   Independente    0 non-null      object
 8   Membro da ONU   0 non-null      object
 9   Status          0 non-null      object
dtypes: object(10)
memory usage: 14.7+ KB


In [22]:
df_onu.shape

(186, 10)

In [23]:
df_onu.columns

Index(['Nome Comum', 'Nome Oficial', 'Código Alpha-2', 'Capital', 'População',
       'Área', 'Continente', 'Independente', 'Membro da ONU', 'Status'],
      dtype='object')

In [24]:
nome_tabela = "onu"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute(f"PRAGMA table_info({nome_tabela});")
columns_info = cursor.fetchall()
for column in columns_info:
    print(column[1])

Nome Comum
Nome Oficial
Código Alpha-2
Capital
População
Área
Continente
Independente
Membro da ONU
Status


## Problema com Dados de Filiados da ONU e Busca na API

Precisei buscar informações mais precisas e atualizadas diretamente na **API oficial da ONU**.


In [25]:
url = 'https://population.un.org/dataportalapi/api/v1/locais'

if response.status_code == 200:
    membros_onu = response.json()
else:
    print(f"Erro na requisição: {response.status_code}")
    print(response.text)

df_membros = pd.DataFrame(membros_onu['data'])
df_membros.columns


Index(['name', 'full_name', 'capital', 'iso2', 'iso3', 'covid19',
       'current_president', 'currency', 'phone_code', 'continent',
       'description', 'size', 'independence_date', 'population', 'href'],
      dtype='object')

In [26]:
url = 'https://population.un.org/dataportalapi/api/v1/locais'
response = requests.get(url)

if response.status_code == 200:
    membros_onu = response.json()
    df_membros = pd.DataFrame(membros_onu['data'])
    df_membros = df_membros[['name']]  


## A NAME corresponde aos dados que nos falta:

**`name`**

In [27]:
coluna_nome = df_membros['name']
coluna_nome


0      Afghanistan
1          Albania
2          Algeria
3          Andorra
4           Angola
          ...     
181      Venezuela
182        Vietnam
183          Yemen
184         Zambia
185       Zimbabwe
Name: name, Length: 186, dtype: object

In [28]:
db_path = os.path.join(endereco_base, "dados_analise.db")

nome_tabela_onu = "onu"

In [29]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{nome_tabela_onu}';")

table_exists = cursor.fetchone()
df_dados_onu = pd.read_sql_query(f"SELECT * FROM {nome_tabela_onu}", conn)
conn.close()    


In [30]:
nome_tabela = "onu"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute(f"PRAGMA table_info({nome_tabela});")
columns_info = cursor.fetchall()
for column in columns_info:
    print(column[1])

Nome Comum
Nome Oficial
Código Alpha-2
Capital
População
Área
Continente
Independente
Membro da ONU
Status


In [31]:
nome_nova_coluna = 'paises_membros_da_ONU'
df_onu[nome_nova_coluna] = coluna_nome

In [32]:
df_onu.head()

,Nome Comum,Nome Oficial,Código Alpha-2,Capital,População,Área,Continente,Independente,Membro da ONU,Status,paises_membros_da_ONU
0,Afghanistan,Islamic Republic of Afghanistan,AF,Kabul,"39,306,195","652,000 km²",Asia,None,None,None,Afghanistan
1,Albania,Republic of Albania | Shqipëria,AL,Tirana,"2,876,490","28,748 km²",Europe,None,None,None,Albania
2,Algeria,Democratic Republic of Algeria,DZ,Algiers,"44,190,030","2,381,741 km²",Africa,None,None,None,Algeria
3,Andorra,Principality of Andorra,AD,Andorra la Vella,"77,317",468 km²,Europe,None,None,None,Andorra
4,Angola,Republic of Angola,AO,Luanda,"33,312,843","1,246,620 km²",Africa,None,None,None,Angola


In [33]:
df_onu.columns

Index(['Nome Comum', 'Nome Oficial', 'Código Alpha-2', 'Capital', 'População',
       'Área', 'Continente', 'Independente', 'Membro da ONU', 'Status',
       'paises_membros_da_ONU'],
      dtype='object')

In [34]:
db_path = 'dados_analise.db'
nome_tabela_onu = "onu"
nome_coluna_paises_membros = 'paises_membros_da_ONU'
nome_coluna_nome_comum = 'Nome Comum'
nome_coluna_membro_onu = 'Membro da ONU'

conn = sqlite3.connect(db_path)
df_onu = pd.read_sql_query(f"SELECT * FROM {nome_tabela_onu}", conn)
nome_nova_coluna = 'paises_membros_da_ONU'

df_onu[nome_nova_coluna] = coluna_nome
print("Colunas após leitura:", df_onu.columns)

def verificar_membro(row):
    nome_comum = row[nome_coluna_nome_comum]
    if nome_coluna_paises_membros in row:
        string_paises_membros = row[nome_coluna_paises_membros]
        if isinstance(string_paises_membros, str) and nome_comum in string_paises_membros:
            return 'YES'
        else:
            return 'NO'
    else:
        return 'Coluna "paises_membros_da_ONU" não encontrada na linha'

df_onu[nome_coluna_membro_onu] = df_onu.apply(verificar_membro, axis=1)

nome_coluna_para_excluir = 'paises_membros_da_ONU'  

df_onu = df_onu.drop(columns=[nome_coluna_para_excluir], axis=1, errors='ignore')
conn.close()

df_onu.head() 


Colunas após leitura: Index(['Nome Comum', 'Nome Oficial', 'Código Alpha-2', 'Capital', 'População',
       'Área', 'Continente', 'Independente', 'Membro da ONU', 'Status',
       'paises_membros_da_ONU'],
      dtype='object')


,Nome Comum,Nome Oficial,Código Alpha-2,Capital,População,Área,Continente,Independente,Membro da ONU,Status
0,Afghanistan,Islamic Republic of Afghanistan,AF,Kabul,"39,306,195","652,000 km²",Asia,None,YES,None
1,Albania,Republic of Albania | Shqipëria,AL,Tirana,"2,876,490","28,748 km²",Europe,None,YES,None
2,Algeria,Democratic Republic of Algeria,DZ,Algiers,"44,190,030","2,381,741 km²",Africa,None,YES,None
3,Andorra,Principality of Andorra,AD,Andorra la Vella,"77,317",468 km²,Europe,None,YES,None
4,Angola,Republic of Angola,AO,Luanda,"33,312,843","1,246,620 km²",Africa,None,YES,None


In [35]:
db_path = 'dados_analise.db'
nome_tabela_onu = "onu"
nome_coluna_paises_membros = 'paises_membros_da_ONU'
nome_coluna_nome_comum = 'Nome Comum'
nome_coluna_membro_onu = 'Membro da ONU'

conn = sqlite3.connect(db_path)
nome_coluna_para_excluir = 'Independente'
df_onu = df_onu.drop(columns=[nome_coluna_para_excluir], axis=1, errors='ignore')
conn.close()


df_onu.head() 



,Nome Comum,Nome Oficial,Código Alpha-2,Capital,População,Área,Continente,Membro da ONU,Status
0,Afghanistan,Islamic Republic of Afghanistan,AF,Kabul,"39,306,195","652,000 km²",Asia,YES,None
1,Albania,Republic of Albania | Shqipëria,AL,Tirana,"2,876,490","28,748 km²",Europe,YES,None
2,Algeria,Democratic Republic of Algeria,DZ,Algiers,"44,190,030","2,381,741 km²",Africa,YES,None
3,Andorra,Principality of Andorra,AD,Andorra la Vella,"77,317",468 km²,Europe,YES,None
4,Angola,Republic of Angola,AO,Luanda,"33,312,843","1,246,620 km²",Africa,YES,None


In [36]:
db_path = 'dados_analise.db'
nome_tabela_onu = "onu"
nome_coluna_paises_membros = 'paises_membros_da_ONU'
nome_coluna_nome_comum = 'Nome Comum'
nome_coluna_membro_onu = 'Membro da ONU'

conn = sqlite3.connect(db_path)
nome_coluna_para_excluir = 'Status'
df_onu = df_onu.drop(columns=[nome_coluna_para_excluir], axis=1, errors='ignore')
conn.close()


df_onu.head() 


,Nome Comum,Nome Oficial,Código Alpha-2,Capital,População,Área,Continente,Membro da ONU
0,Afghanistan,Islamic Republic of Afghanistan,AF,Kabul,"39,306,195","652,000 km²",Asia,YES
1,Albania,Republic of Albania | Shqipëria,AL,Tirana,"2,876,490","28,748 km²",Europe,YES
2,Algeria,Democratic Republic of Algeria,DZ,Algiers,"44,190,030","2,381,741 km²",Africa,YES
3,Andorra,Principality of Andorra,AD,Andorra la Vella,"77,317",468 km²,Europe,YES
4,Angola,Republic of Angola,AO,Luanda,"33,312,843","1,246,620 km²",Africa,YES


In [37]:
df_onu['Membro da ONU'].value_counts()

Membro da ONU
YES    186
Name: count, dtype: int64